In [ ]:
#
# Prototyping parts of the following:
#
# Drawing Graphs to Convey Proximity: An Incremental Arrangement Method
# J.D. Cohen
# ACM Transactions on Computer-Human Interaction, Vol. 4, No. 3, September 1997, Pages 197–229.
#
import polars as pl
import networkx as nx

from math import log10, ceil
import random
import time
import rtsvg
from rtsvg.polars_force_directed_layout import PolarsForceDirectedLayout
rt = rtsvg.RACETrack()

from linknode_graph_patterns import LinkNodeGraphPatterns
_patterns_ = LinkNodeGraphPatterns()

def graphAsDataFrame(_g_):
    _lu_ = {'fm':[], 'to':[]}
    for _e_ in _g_.edges: _lu_['fm'].append(_e_[0]), _lu_['to'].append(_e_[1])
    return pl.DataFrame(_lu_)

#
# Attempting to match Figure 1 from "Drawing Graphs To Convey Proximity"
# - looks like i don't do enough iterations for proportional or semi-proportional
# - absolute stress looks like the papers version
#
_tiles_ = []
g       = _patterns_.createPattern('X')
df      = graphAsDataFrame(g)
for k in range(3):
    _lu_ = {'iteration':[], 'stress':[], 'trial':[], 'time':[]}
    for _trial_ in range(1): # should be 15 to 25
        t0    = time.time()
        pfdl  = rtsvg.PolarsForceDirectedLayout(g, k=k)
        t1    = time.time()
        _vec_ = pfdl.stressVector()
        for i in range(len(_vec_)):
            _lu_['iteration'].append(i), _lu_['stress'].append(log10(_vec_[i])), _lu_['trial'].append(_trial_), _lu_['time'].append(t1-t0)
    df_stress = pl.DataFrame(_lu_)
    _tiles_.append(rt.titleSVG(rt.xy(df_stress, x_field='iteration', y_field='stress', line_groupby_field='trial', dot_size=None), f'k={k}'))
#rt.tile(_tiles_, spacer=10) # uncomment to display

In [ ]:
def __getTargetDistances__(_g_): return dict(nx.all_pairs_dijkstra_path_length(_g_))

# Table IV of paper
def __everyNthMember__(Q, n): return [Q[i] for i in range(0, len(Q), n)]
def __disperseTheseVertices__(Q, increment_ratio=1):
    if len(Q) > increment_ratio + 1:
        F = __everyNthMember__(Q, 1+increment_ratio)
        B = [item for item in Q if item not in F]     # Q - F
        F = __disperseTheseVertices__(F)
        F.extend(B)
        F
    return Q

def __orderVertices__(_g_, _dist_):
    Q = [n for n in nx.traversal.dfs_preorder_nodes(_g_)]
    return __disperseTheseVertices__(Q)

# Table III of paper
def __numberToAddThisTime__(_prev_, _final_, increment_ratio=1, increment_minimum=10):
    if _prev_ > 0: _inc_ = _prev_ * increment_ratio
    else:
        _inc_ = _final_
        while _inc_ > increment_minimum: _inc_ = ceil(_inc_ / (1 + increment_ratio))
    if _inc_ > _final_ - _prev_: _inc_ = _final_ - _prev_
    return _inc_

# Primitive version of closest members
def __closestMembers__(_H_, _v_, _distances_):
    _h1_, _h1_d_, _h2_, _h2_d_ = None, None, None, None
    for _k_ in _H_:
        if _k_ == _v_: continue
        _d_ = _distances_[_v_][_k_]
        if   _h1_d_ is None: _h1_, _h1_d_ = _k_, _d_
        elif _h2_d_ is None: _h2_, _h2_d_ = _k_, _d_
        elif _d_ < _h1_d_ or _d_ < _h2_d_:
            if   _d_ < _h1_d_ and _d_ < _h2_d_:
                if _h1_d_ < _h2_d_: _h2_, _h2_d_ = _k_, _d_
                else:               _h1_, _h1_d_ = _k_, _d_
            elif _d_ < _h1_d_:      _h1_, _h1_d_ = _k_, _d_
            else:                   _h2_, _h2_d_ = _k_, _d_
    return _h1_, _h2_

# Figure 3 from the paper (and the accompanying formulas)
# Primitive version of neighborlyLocation()
def __neighborlyLocation__(i, j, k, _pos_, _distances_):
    t_ik, t_ij, t_jk = _distances_[i][k], _distances_[i][j], _distances_[j][k]
    _expr_   = (1.0 / (2 * t_jk**2)) * (t_ik**2 - t_ij**2 - t_jk**2)
    _gamma_  = min(_expr_, 0.5)
    x_j, y_j = _pos_[j]
    x_k, y_k = _pos_[k]
    e_x, e_y = random.random() * 0.01, random.random() * 0.01
    x, y     = x_j + _gamma_ * (x_j - x_k) + e_x, y_j + _gamma_ * (y_j - y_k) + e_y
    return x, y

#
# Modified version from the PolarsForceDirectedLayout implementation
#
def __arrangeDirect__(_nodes_, _pos_, _distances_, k):
    # Construct the df_pos and df_dist dataframes
    _lu_pos_  = {'node':[], 'x':[], 'y':[], 's':[]}
    _lu_dist_ = {'fm':[],'to':[], 't':[]}
    for _node_ in _nodes_:
        _lu_pos_['node'].append(_node_), _lu_pos_['x'].append(_pos_[_node_][0]), _lu_pos_['y'].append(_pos_[_node_][1]), _lu_pos_['s'].append(False)
        for _nbor_ in _nodes_:
            if _nbor_ == _node_: continue
            _lu_dist_['fm'].append(_node_), _lu_dist_['to'].append(_nbor_), _lu_dist_['t'].append(_distances_[_node_][_nbor_])
    df_pos, df_dist = pl.DataFrame(_lu_pos_), pl.DataFrame(_lu_dist_)

    # Iterate using the force directed layout algorithm
    iterations = min(64, 2*len(_nodes_))
    mu         = 1.0/(2.0*len(_nodes_))
    __dx__, __dy__ = (pl.col('x') - pl.col('x_right')), (pl.col('y') - pl.col('y_right'))
    for i in range(iterations):
        df_pos = df_pos.join(df_pos, how='cross') \
                       .filter(pl.col('node') != pl.col('node_right')) \
                       .with_columns((__dx__**2 + __dy__**2).sqrt().alias('d')) \
                       .join(df_dist, left_on=['node', 'node_right'], right_on=['fm','to']) \
                       .with_columns(pl.col('t').pow(k).alias('t_k')) \
                       .with_columns(pl.when(pl.col('d') < 0.001).then(pl.lit(0.001)).otherwise(pl.col('d')).alias('d'),
                                     pl.when(pl.col('t') < 0.001).then(pl.lit(0.001)).otherwise(pl.col('t')).alias('w')) \
                       .with_columns(((2.0*__dx__*(1.0 - pl.col('t')/pl.col('d')))/pl.col('t_k')).alias('xadd'),
                                     ((2.0*__dy__*(1.0 - pl.col('t')/pl.col('d')))/pl.col('t_k')).alias('yadd'),
                                     (((pl.col('t') - pl.col('d'))**2)/pl.col('t_k')).alias('stress')) \
                       .group_by(['node','x','y','s']).agg(pl.col('xadd').sum(), pl.col('yadd').sum(), pl.col('stress').sum()/len(_nodes_)) \
                       .with_columns(pl.when(pl.col('s')).then(pl.col('x')).otherwise(pl.col('x') - mu * pl.col('xadd')).alias('x'),
                                     pl.when(pl.col('s')).then(pl.col('y')).otherwise(pl.col('y') - mu * pl.col('yadd')).alias('y')) \
                       .drop(['xadd','yadd'])
    _updated_ = {}
    for i in range(len(df_pos)): _updated_[df_pos['node'][i]] = (df_pos['x'][i], df_pos['y'][i])
    return _updated_

#
# Table II of paper
#
def arrangeIncrementally(g_connected, k=2.0):
    pos         = {}                                                   # Results
    V           = set(g_connected.nodes)
    distances   = __getTargetDistances__(g_connected)                  # Establish target distances
    Q           = __orderVertices__(g_connected, distances)            # Make vector Q of vertices in order of inclusion
    H           = set()                                                # Initialize vertices to arrange

    # There needs to be an initial vertex set that is already placed for the while loop to actually work...
    # vvv
    for i in range(__numberToAddThisTime__(len(H), len(V))):
        H.add(Q[0])
        pos[Q[0]] = (random.random(), random.random())
        Q = Q[1:]
    pos = __arrangeDirect__(H, pos, distances, k=k)
    # ^^^

    while H != V:
        _number_to_add_ = __numberToAddThisTime__(len(H), len(V))
        for i in range(_number_to_add_):
            v      = Q[0]                                              # Get next vertex
            h1, h2 = __closestMembers__(H, v, distances)               # Find closest two members of H
            pos[v] = __neighborlyLocation__(v, h1, h2, pos, distances) # Put new vertex near them
            Q      = Q[1:]                                             # This vertex is done
            H.add(v)
        pos = __arrangeDirect__(H, pos, distances, k=k)                # Arrange accumulated subset
    return pos

rt.link(df, [('fm','to')], arrangeIncrementally(g))